Hi Buddy!<br> well for this exercise we will be following an <a ref='https://www.kdnuggets.com/guide-deploying-machine-learning-models-production'>article</a>.

First open your terminal and create an virtual environment.
> python -m venv myvirtualenv

Create the `requirements.txt` file and fill it out with the following library list.
<br>

pandas
scikit-learn
fastapi
pydantic
uvicorn
streamlit
